In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from requests.auth import HTTPBasicAuth
import numpy as np

import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from datetime import datetime
import time 
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

### Function to scrape Amazon

In [ ]:
def scrape_amazon():
    
    timestamp = datetime.now().isoformat()
    timestamp = timestamp.split('T')
    timestamp[1] = timestamp[1].split('.')[0]

    amazon_product_info = []

    query = input("Please enter an item you are looking for...")

    options = Options()
    options.add_argument("--headless")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get("https://www.amazon.com")
    time.sleep(3)

    for page_number in range (1,7):
        url = f"https://www.amazon.com/s?k={query}&page={page_number}"
        driver.get(url)
        
        body = driver.find_element(By.TAG_NAME, "body")
        for i in range(5):
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(1)

        html = driver.page_source
        soup = BeautifulSoup(html)
        products = soup.find_all("div", attrs={"data-asin": True, "data-component-type": "s-search-result"})

        for product in products:
            product_info_amazon = {}

            name_tag = product.find('h2', attrs={"class": re.compile("a-size-base-plus a-spacing-none a-color-base a-text-normal|a-size-medium a-spacing-none a-color-base a-text-normal")})
            if name_tag:
                product_info_amazon['name'] = name_tag.get_text(strip=True)
            else:
                continue

            price_tag = product.find('span', attrs={"class": "a-offscreen"})
            if price_tag:
                product_info_amazon['price'] = price_tag.get_text(strip=True)
            else:
                product_link_tag = product.find("a", attrs={"class": "a-link-normal s-line-clamp-2 s-link-style a-text-normal"})
                if product_link_tag:
                    product_url = "https://www.amazon.com" + product_link_tag['href']
                    driver.get(product_url)
                    time.sleep(2)

                    product_page_soup = BeautifulSoup(driver.page_source)

                    inside_price_tag = product_page_soup.find("span", attrs={"class": "a-price aok-align-center reinventPricePriceToPayMargin priceToPay"})
                    if inside_price_tag:
                        product_info_amazon['price'] = inside_price_tag.get_text(strip=True)
                    else:
                        continue
                else:
                    continue
            
            rating_tag = product.find('span', attrs= {"class": "a-icon-alt"})
            if rating_tag:
                product_info_amazon['rating'] = rating_tag.get_text(strip=True)
            else:
                None

            amount_reviews_tag = product.find('span', attrs= {"class": "a-size-base s-underline-text"})
            if amount_reviews_tag:
                product_info_amazon['amount_of_reviews'] = amount_reviews_tag.get_text(strip=True)
            else:
                None

            url_tag = product.find("a", attrs={"class": "a-link-normal s-line-clamp-2 s-link-style a-text-normal"}) or product.find("a", attrs={"class": "a-link-normal s-no-outline"})
            if url_tag and 'href' in url_tag.attrs:
                product_info_amazon['link'] = "https://www.amazon.com" + url_tag['href']


            deliver_tag = product.find('span', attrs= {"class": "a-color-base a-text-bold"})
            if deliver_tag:
                product_info_amazon['delivery'] = deliver_tag.get_text(strip=True)
            else:
                product_link_tag = product.find("a", attrs={"class": "a-link-normal s-line-clamp-2 s-link-style a-text-normal"})
                if product_link_tag:
                    product_url = "https://www.amazon.com" + product_link_tag['href']
                    driver.get(product_url)
                    time.sleep(2)

                    product_page_soup = BeautifulSoup(driver.page_source)

                    delivery_tags = product_page_soup.find_all("span", class_="a-text-bold")
                    for tag in delivery_tags:
                        text = tag.get_text(strip=True)
                        if "," in text:
                            product_info_amazon["delivery"] = text
                        break
                    else:
                        continue
                else:
                    continue
            
            bought_tags = product.find_all('span', attrs={"class": "a-size-base a-color-secondary"})
            for tag in bought_tags:
                text = tag.get_text(strip=True)
                if "bought in past month" in text:
                    product_info_amazon['bought_so_far'] = text
                    break

            if product_info_amazon.get('link'):
                driver.get(product_info_amazon['link'])
                time.sleep(2)

                product_page_soup = BeautifulSoup(driver.page_source, 'html.parser')

                seller = product_page_soup.select_one("#sellerProfileTriggerId")
                product_info_amazon['seller'] = seller.get_text(strip=True)

                refund_tags = product_page_soup.find_all('span', attrs={"class": "a-size-small offer-display-feature-text-message"})
                product_info_amazon['return_policy'] = refund_tags[4].get_text(strip=True) if len(refund_tags) > 4 else None

                shipping_tags = product_page_soup.find_all("span", attrs={"class": "a-size-base a-color-secondary"})
                for tag in shipping_tags:
                    text = tag.get_text(strip=True)
                    if "Shipping" in text or "Delivery" in text or "$" in text:
                        shipping_price = text
                        break
                product_info_amazon['shipping'] = shipping_price if shipping_price else "None"

            product_info_amazon['time_of_scraping'] = timestamp
            amazon_product_info.append(product_info_amazon)
            
    driver.quit()
    df = pd.DataFrame(amazon_product_info)
    df.to_csv('amazon_nikon.csv', index = False)
    return df

### Function to scrape e-bay

In [3]:
def scrape_ebay():
    timestamp = datetime.now().isoformat()
    timestamp = timestamp.split('T')
    timestamp[1] = timestamp[1].split('.')[0]

    ebay_product_info = []

    query = input("Please enter an item you are looking for...")

    options = Options()
    options.add_argument("--headless")
        
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get("https://www.ebay.com/")
    time.sleep(3)

    for page_number in range(1, 4):
        driver.get(f"https://www.ebay.com/sch/i.html?_nkw={query}&_sacat=0&_from=R40&_pgn={page_number}")
        time.sleep(5)

        html = driver.page_source
        soup = BeautifulSoup(html)
        products = soup.find_all("li", attrs={"class": "s-item s-item__pl-on-bottom"})

        body = driver.find_element(By.TAG_NAME, "body")
        for i in range(5):
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(1)

        for product in products:
            product_info_ebay = {}

            name_tag = product.find('div', attrs={"class": "s-item__title"})
            if name_tag:
                name_text = name_tag.get_text(strip=True)
                if "Shop on eBay" in name_text:
                    continue
                if "New Listing" in name_text:
                    replaced = name_text.replace("New Listing", "").strip()
                    product_info_ebay['name'] = replaced
                else:
                    product_info_ebay['name'] = name_text
            else:
                continue

            price_tag = product.find('span', attrs={"class": "s-item__price"})
            if price_tag:
                product_info_ebay['price'] = price_tag.get_text(strip=True)

            rating_tag = product.find("div", attrs={"class": "x-star-rating"})
            if rating_tag:
                product_info_ebay['rating'] = rating_tag.get_text(strip=True)
            else:
                product_info_ebay['rating'] = 'No reviews'

            reviews_tag = product.select_one("span.s-item__reviews-count span")
            product_info_ebay['amount_of_reviews'] = reviews_tag.get_text(strip=True) if reviews_tag else 'No reviews'

            urls = product.find("a", attrs= {"class": "s-item__link"})
            if urls:
                product_info_ebay['link'] = urls['href']
            else:
                product_info_ebay['link'] = 'No URL'

            sold_tag = product.find("span", attrs={"class": "s-item__dynamic s-item__quantitySold"})
            if sold_tag:
                product_info_ebay['sold'] = sold_tag.get_text(strip=True)
            else:
                product_info_ebay['sold'] = 'None'

            seller_tag = product.find("span", attrs={"class": "s-item__seller-info-text"})
            if seller_tag:
                seller = seller_tag.get_text(strip=True).split(" ")[0].replace("'", "")
                product_info_ebay['seller'] = seller
            else:
                product_info_ebay['seller'] = 'No information'

            if product_info_ebay['link']:
                driver.get(product_info_ebay['link'])
                time.sleep(3)

                shipping_divs = driver.find_elements(By.CLASS_NAME, "ux-labels-values__values-content")
                shipping_texts = [div.text for div in shipping_divs]
                match_found = False
                for text in shipping_texts:
                    if "US" or "EUR" in text:
                        product_info_ebay['shipping'] = text
                        match_found = True
                        break
                if not match_found:
                    product_info_ebay['shipping'] = None 

                delivery_divs = driver.find_elements(By.CLASS_NAME, "ux-labels-values__values-content")
                delivery_texts = [div.text for div in delivery_divs]
                match_found = False
                for text in delivery_texts:
                    if "Estimated between" in text:
                        product_info_ebay['delivery'] = text
                        match_found = True
                        break
                if not match_found:
                    product_info_ebay['delivery'] = None

                refund_divs = driver.find_elements(By.CLASS_NAME, "ux-labels-values__values-content")
                refund_texts = [div.text for div in refund_divs]
                match_found = False
                for text in refund_texts:
                    if "returns" in text:
                        product_info_ebay['refund'] = text
                        match_found = True
                        break
                if not match_found:
                    product_info_ebay['refund'] = None

            product_info_ebay['time_of_scrape'] = timestamp
            ebay_product_info.append(product_info_ebay)

    driver.quit()
    df = pd.DataFrame(ebay_product_info)
    df.to_csv('ebay_nikon.csv', index=False)
    return

### Structuring Data Frames & Cleaning scraped data from both platforms

In [4]:
path1 = "/Users/rafalkolakowski/Desktop/DA/Week 8/final_project/scrapings/amazon_kitchen.csv"
dfamazonkitchen = pd.read_csv(path1)

path2 = "/Users/rafalkolakowski/Desktop/DA/Week 8/final_project/scrapings/amazon_tshirt.csv"
dfamazontshirt = pd.read_csv(path2)

path3 = "/Users/rafalkolakowski/Desktop/DA/Week 8/final_project/scrapings/ebay_kitchen.csv"
dfebaykitchen = pd.read_csv(path3)

path4 = "/Users/rafalkolakowski/Desktop/DA/Week 8/final_project/scrapings/ebay_tshirt.csv"
dfebaytshirt = pd.read_csv(path4)

path5 = "/Users/rafalkolakowski/Desktop/DA/Week 8/final_project/scrapings/amazon_nikon.csv"
dfamazonnikon = pd.read_csv(path5)

path6 = "/Users/rafalkolakowski/Desktop/DA/Week 8/final_project/scrapings/ebay_nikon.csv"
dfebaynikon = pd.read_csv(path6)

In [5]:
dfebaynikon['platform'] = "ebay"
dfebaykitchen['platform'] = "ebay"
dfebaytshirt['platform'] = "ebay"

In [6]:
dfamazonkitchen['platform'] = 'amazon'
dfamazonnikon['platform'] = 'amazon'
dfamazontshirt['platform'] = 'amazon'

In [7]:
dfebaynikon.rename(columns={
    "refund": "return_policy",
    "time_of_scrape": "time_of_scraping",
    "sold": "bought_so_far"
}, inplace=True)

dfebaytshirt.rename(columns={
    "refund": "return_policy",
    "time_of_scrape": "time_of_scraping",
    "sold": "bought_so_far"
}, inplace=True)

dfebaykitchen.rename(columns={
    "refund": "return_policy",
    "time_of_scrape": "time_of_scraping",
    "sold": "bought_so_far"
}, inplace=True)

In [8]:
scraped_df = pd.concat([
    dfamazonnikon,
    dfamazonkitchen,
    dfamazontshirt,
    dfebaykitchen,
    dfebaynikon,
    dfebaytshirt], join='inner', ignore_index=True)

In [9]:
scraped_df = scraped_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [10]:
scraped_df["price"] = scraped_df["price"].str.replace("$", "")
scraped_df["price"] = scraped_df["price"].str.replace(",", "")
scraped_df["price"] = pd.to_numeric(scraped_df["price"], errors = "coerce")
scraped_df["price"] = scraped_df["price"].astype(float)

In [11]:
def conver_usd_to_eur(amount_usd, rate=0.93):
    return round(amount_usd * rate, 2)

In [12]:
scraped_df["price"].apply(conver_usd_to_eur)
scraped_df.rename(columns={"price": "price_eur"}, inplace=True)

In [13]:
scraped_df.dropna(subset=["price_eur"], inplace=True, ignore_index=True)
scraped_df["price_eur"].isnull().sum()

0

In [14]:
scraped_df.dtypes

name                  object
price_eur            float64
rating                object
amount_of_reviews     object
link                  object
delivery              object
bought_so_far         object
seller                object
return_policy         object
shipping              object
time_of_scraping      object
platform              object
dtype: object

In [15]:
def clear_ratings(value):
    if "out of" in str(value):
        return float(value.split(" ")[0])
    else:
        return 0

In [16]:
scraped_df["rating"] = scraped_df["rating"].apply(clear_ratings)

In [17]:
def clean_amount_reviews(value):
    value = str(value).lower().strip()
    
    value = value.replace(",", "")    
    parts = value.split(" ")
    
    if parts[0].isdigit():
        return int(parts[0])
    
    return 0

In [18]:
scraped_df["amount_of_reviews"] = scraped_df["amount_of_reviews"].apply(clean_amount_reviews)

In [19]:
scraped_df["time_of_scraping"] = scraped_df["time_of_scraping"].str.extract(r"'(\d{4}-\d{2}-\d{2})'")
scraped_df["scraping_date"] = pd.to_datetime(scraped_df["time_of_scraping"], errors="coerce")
scraped_df.drop(columns="time_of_scraping", inplace=True)


In [20]:
scraped_df

,name,price_eur,rating,amount_of_reviews,link,delivery,bought_so_far,seller,return_policy,shipping,platform,scraping_date
0,Men's UPF 50+ Sun Protection Athletic Shirts S...,17.99,4.6,23,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,"Wed, Apr 30",NaN,PAUSEL,30-day refund/replacement,No Import Fees Deposit & $9.58 Shipping to Ger...,amazon,2025-04-14
1,"Jazz 45 v2.0 Camera Bag for Photographers, Cam...",44.99,4.0,34,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,"Wed, Apr 30",NaN,Tamrac Inc.,30-day refund/replacement,No Import Fees Deposit & $12.04 Shipping to Ge...,amazon,2025-04-14
2,Kitchen Stuff Bonds,319.00,0.0,0,https://www.ebay.com/itm/356115048000?_skw=kit...,"Estimated between Mon, May 5 and Wed, May 14 t...",NaN,heca20,30 days returns. Buyer pays for return shippin...,US $31.04 eBay International Shipping\n. See d...,ebay,2025-04-14
3,MLB City Connect Wordmark T-Shirt,39.99,1.0,1,https://www.amazon.com/Nike-Wordmark-Performan...,"Fri, May 2",NaN,NaN,NaN,NaN,amazon,2025-04-14
4,"Paper Towel Holder Countertop, One-Handed Tear...",16.99,4.6,1672,https://www.amazon.com/Swaitee-Countertop-Mech...,"Wed, Apr 30",6K+ bought in past month,Swaitee,30-day refund/replacement,No Import Fees Deposit & $12.46 Shipping to Ge...,amazon,2025-04-14
...,...,...,...,...,...,...,...,...,...,...,...,...
1311,Nikon D7500 DSLR Camera Body & Lens' Boxed Nik...,1505.01,0.0,0,https://www.ebay.com/itm/156435148545?_skw=Nik...,"Estimated between Mon, Apr 21 and Fri, Apr 25 ...",NaN,mikethetrader_9,Seller does not accept returns. See details\n-...,GBP 74.03 (approx US $96.88) International Pri...,ebay,2025-04-14
1312,Nikon D D7500 20.9MP Digital SLR Camera - Blac...,749.00,5.0,18,https://www.ebay.com/itm/226654507557?_skw=Nik...,No information,NaN,aonetradeltd,14 days returns. Buyer pays for return shippin...,May not ship to Germany. Read item description...,ebay,2025-04-14
1313,Nike Men's Sole 3 HBR Food Graphic T-Shirt in ...,22.00,0.0,0,https://www.ebay.com/itm/276151543917?_skw=nik...,"Estimated between Mon, May 5 and Mon, May 12 t...",NaN,No information,30 days returns. Buyer pays for return shippin...,US $12.81 eBay International Shipping\n. See d...,ebay,2025-04-14
1314,Let's Bake Stuff And Watch Christmas Movies Ov...,6.80,0.0,0,https://www.ebay.com/itm/315734497230?_skw=kit...,"Estimated between Tue, Apr 22 and Mon, May 5 t...",NaN,crazydogtshirts,30 days returns. Buyer pays for return shippin...,NaN,ebay,2025-04-14


In [21]:
def clean_delivery(text):
    text = str(text)

    matches = re.findall(r"(?:[A-Z][a-z]{2},\s)?([A-Z][a-z]{2})\s(\d{1,2})", text)
    if matches:
        month, day = matches[-1]
        try:
            return datetime.strptime(f"{month} {day} 2025", "%b %d %Y")
        except:
            return "No delivery information"
    return "No delivery information"

In [22]:
scraped_df["delivery_clean"] = scraped_df["delivery"].apply(clean_delivery)
scraped_df["delivery_clean"] = pd.to_datetime(scraped_df["delivery_clean"], errors="coerce")
scraped_df["delivery_days"] = (scraped_df["delivery_clean"] - scraped_df["scraping_date"]).dt.days

In [23]:
scraped_df.drop(columns="delivery", inplace=True)

In [24]:
scraped_df["shipping"]

0       No Import Fees Deposit & $9.58 Shipping to Ger...
1       No Import Fees Deposit & $12.04 Shipping to Ge...
2       US $31.04 eBay International Shipping\n. See d...
3                                                     NaN
4       No Import Fees Deposit & $12.46 Shipping to Ge...
                              ...                        
1311    GBP 74.03 (approx US $96.88) International Pri...
1312    May not ship to Germany. Read item description...
1313    US $12.81 eBay International Shipping\n. See d...
1314                                                  NaN
1315    US $50.00 FedEx Ground or FedEx Home Delivery®...
Name: shipping, Length: 1316, dtype: object

In [25]:
def extract_shipping_price(text):
    text = str(text)

    match = re.search(r"(?:USD|US)?\s?\$?£?€?(?:GBP)?\s?(\d+\.\d{2})", text)
    if match:
        return float(match.group(1))
    return "No information provided"

In [26]:
scraped_df["shipping_eur"] = scraped_df["shipping"].apply(extract_shipping_price)
scraped_df.drop(columns="shipping", inplace=True)

In [27]:
scraped_df["bought_so_far"].value_counts()

bought_so_far
50+ bought in past month     77
100+ bought in past month    66
1K+ bought in past month     49
2K+ bought in past month     33
200+ bought in past month    32
300+ bought in past month    26
10K+ bought in past month    23
3K+ bought in past month     20
400+ bought in past month    19
500+ bought in past month    18
4K+ bought in past month     14
5K+ bought in past month     13
900+ bought in past month     9
600+ bought in past month     8
7K+ bought in past month      6
800+ bought in past month     6
6K+ bought in past month      6
8K+ bought in past month      6
700+ bought in past month     5
20K+ bought in past month     5
9K+ bought in past month      4
62+ sold                      2
50K+ bought in past month     2
60K+ bought in past month     1
47 sold                       1
30K+ bought in past month     1
13+ sold                      1
11+ sold                      1
76 sold                       1
64 sold                       1
29 sold                   

In [28]:
def extract_bought_amount(text):
    text = str(text)
    match = re.search(r"(\d+)(K)?", text)
    if match:
        num = int(match.group(1))
        if match.group(2) == "K":
            return num * 1000
        return num
    return "No buyers"

In [29]:
scraped_df["bought_so_far"] = scraped_df["bought_so_far"].apply(extract_bought_amount)

In [30]:
scraped_df["return_policy"].value_counts()

return_policy
30-day refund/replacement                                                                                                                                                                                                    598
30 days returns. Buyer pays for return shipping. If you use an eBay shipping label, it will be deducted from your refund amount. See details\n- for more information about returns                                           305
Seller does not accept returns. See details\n- for more information about returns                                                                                                                                             89
30 days returns. Seller pays for return shipping. See details\n- for more information about returns                                                                                                                           56
60 days returns. Buyer pays for return shipping. If you use an eBay shipping label, it

In [31]:
def simplify_return_policy(text):
    text = str(text).lower()

    if "does not accept returns" in text:
        return "no returns"
    elif "30-day" in text or "30 day" in text:
        return "30 days"
    elif "60-day" in text or "60 day" in text:
        return "60 days"
    elif "90-day" in text or "90 day" in text:
        return "90 days"
    elif "14-day" in text or "14 day" in text:
        return "14 days"
    else:
        return None

In [32]:
scraped_df

,name,price_eur,rating,amount_of_reviews,link,bought_so_far,seller,return_policy,platform,scraping_date,delivery_clean,delivery_days,shipping_eur
0,Men's UPF 50+ Sun Protection Athletic Shirts S...,17.99,4.6,23,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,No buyers,PAUSEL,30-day refund/replacement,amazon,2025-04-14,2025-04-30,16.0,9.58
1,"Jazz 45 v2.0 Camera Bag for Photographers, Cam...",44.99,4.0,34,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,No buyers,Tamrac Inc.,30-day refund/replacement,amazon,2025-04-14,2025-04-30,16.0,12.04
2,Kitchen Stuff Bonds,319.00,0.0,0,https://www.ebay.com/itm/356115048000?_skw=kit...,No buyers,heca20,30 days returns. Buyer pays for return shippin...,ebay,2025-04-14,2025-05-14,30.0,31.04
3,MLB City Connect Wordmark T-Shirt,39.99,1.0,1,https://www.amazon.com/Nike-Wordmark-Performan...,No buyers,NaN,NaN,amazon,2025-04-14,2025-05-02,18.0,No information provided
4,"Paper Towel Holder Countertop, One-Handed Tear...",16.99,4.6,1672,https://www.amazon.com/Swaitee-Countertop-Mech...,6000,Swaitee,30-day refund/replacement,amazon,2025-04-14,2025-04-30,16.0,12.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,Nikon D7500 DSLR Camera Body & Lens' Boxed Nik...,1505.01,0.0,0,https://www.ebay.com/itm/156435148545?_skw=Nik...,No buyers,mikethetrader_9,Seller does not accept returns. See details\n-...,ebay,2025-04-14,2025-04-25,11.0,74.03
1312,Nikon D D7500 20.9MP Digital SLR Camera - Blac...,749.00,5.0,18,https://www.ebay.com/itm/226654507557?_skw=Nik...,No buyers,aonetradeltd,14 days returns. Buyer pays for return shippin...,ebay,2025-04-14,NaT,NaN,No information provided
1313,Nike Men's Sole 3 HBR Food Graphic T-Shirt in ...,22.00,0.0,0,https://www.ebay.com/itm/276151543917?_skw=nik...,No buyers,No information,30 days returns. Buyer pays for return shippin...,ebay,2025-04-14,2025-05-12,28.0,12.81
1314,Let's Bake Stuff And Watch Christmas Movies Ov...,6.80,0.0,0,https://www.ebay.com/itm/315734497230?_skw=kit...,No buyers,crazydogtshirts,30 days returns. Buyer pays for return shippin...,ebay,2025-04-14,2025-05-05,21.0,No information provided


In [33]:
scraped_df["return_policy"] = scraped_df["return_policy"].apply(simplify_return_policy)

In [34]:
scraped_df.columns.tolist()

['name',
 'price_eur',
 'rating',
 'amount_of_reviews',
 'link',
 'bought_so_far',
 'seller',
 'return_policy',
 'platform',
 'scraping_date',
 'delivery_clean',
 'delivery_days',
 'shipping_eur']

### Function to scrape empik

In [ ]:
def scrape_empik():
    timestamp = datetime.now().isoformat()
    timestamp = timestamp.split('T')
    timestamp[1] = timestamp[1].split('.')[0]

    query = input("Please enter an item you are looking for...")

    options = Options()
    options.add_argument("--headless")
        
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()

    collected_links = []

    for crop in range(0, 180, 60):
        driver.get(f"https://www.empik.com/szukaj/produkt?start={crop}&q={query}&qtype=basicForm")
        time.sleep(5)

        try:
            button = driver.find_element(By.CSS_SELECTOR, "button.css-18n58r")
            button.click()
            time.sleep(5)
        except:
            pass

        body = driver.find_element(By.TAG_NAME, "body")
        for i in range(5):
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(1)

        products = driver.find_elements(By.CSS_SELECTOR, "div.search-list-item.js-reco-product.js-energyclass-product.ta-product-tile")

        for product in products:
            link_element = product.find_element(By.TAG_NAME, "a")
            product_url = link_element.get_attribute("href")
            collected_links.append({"link": product_url})

        if len(collected_links) >= 200:
            break

    empik_product_info = []

    for link_dict in collected_links:
        driver.get(link_dict['link'])
        time.sleep(5)

        product_data = {}
        product_data["link"] = link_dict["link"]

        try:
            title = driver.find_element(By.CSS_SELECTOR, "h1.css-9rngxa-title").text
            product_data["name"] = title
        except:
            pass
        try:
            price = driver.find_element(By.CSS_SELECTOR, "span.css-1f0k13e-DesktopPriceStyles").text
            product_data["price"] = price
        except:
            pass
        try:
            rating = driver.find_element(By.CSS_SELECTOR, "span.css-1p6necc-score").text
            product_data["rating"] = rating
        except:
            pass   
        try:
            amount_of_reviews = driver.find_element(By.CSS_SELECTOR, "a.css-g50wls-ratingCount").text
            product_data["amount_of_reviews"] = amount_of_reviews
        except:
            pass
        try:
            seller = driver.find_element(By.CSS_SELECTOR, "span.css-1tdlp0o-name").text
            product_data["seller"] = seller
        except:
            pass
        try:
            return_policy = driver.find_element(By.CSS_SELECTOR, "span.css-mnqlmx-ReturnItemStyles").text
            product_data["return_policy"] = return_policy
        except:
            pass
        try:
            delivery_days = driver.find_element(By.CSS_SELECTOR, "span.css-1x8akhw-availabilityName-content-DesktopPriceAndAvailability").text
            product_data["delivery_days"] = delivery_days
        except:
            pass
        try:
            shipping_button = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Dostawa') and contains(text(), 'płatność')]")))
            driver.execute_script("arguments[0].click();", shipping_button)
            WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.body-price-delivery")))
            all_prices = driver.find_elements(By.CSS_SELECTOR, "span.body-price-delivery")
            shipping_eur = all_prices[-2].text
            product_data["shipping_eur"] = shipping_eur
        except: pass

        product_data["date_of_scrape"] = timestamp
        empik_product_info.append(product_data)

    driver.quit()
    df = pd.DataFrame(empik_product_info)
    df.to_csv('empik_kitchen.csv', index=False)
    return

In [36]:
path7 = "/Users/rafalkolakowski/Desktop/DA/Week 8/final_project/scrapings/empik_nikon.csv"
dfempiknikon = pd.read_csv(path7)

path8 = "/Users/rafalkolakowski/Desktop/DA/Week 8/final_project/scrapings/empik_tshirt.csv"
dfempiktshirt = pd.read_csv(path8)

path9 = "/Users/rafalkolakowski/Desktop/DA/Week 8/final_project/scrapings/empik_kitchen.csv"
dfempikkitchen = pd.read_csv(path9)

In [37]:
df_empik = pd.concat([dfempikkitchen, dfempiknikon, dfempiktshirt], join="inner", ignore_index=True)

### Adjusting price & other cleaning queries

In [38]:
def pln_eur_converter(value):
    try:
        return round(float(value) / 4.29, 2)
    except (ValueError, TypeError):
        return value

In [39]:
df_empik["price"] = df_empik["price"].str.replace(",", ".")
df_empik["price"] = df_empik["price"].str.replace("zł", "")
df_empik["price"] = df_empik["price"].astype(float)
df_empik["price"] = df_empik["price"].apply(pln_eur_converter)
df_empik.rename(columns={"price": "price_eur"}, inplace=True)

In [40]:
df_empik

,link,name,price_eur,rating,amount_of_reviews,seller,return_policy,delivery_days,shipping_eur,date_of_scrape
0,https://www.empik.com/rockle-butelka-tritanowa...,ROCKLE BUTELKA TRITANOWA KOSMOS,9.32,"0,0",(0),Empik,do 14 dni,Przewidywana wysyłka w 1 dzień rob.,"15,99 zł","['2025-04-16', '11:49:30']"
1,https://www.empik.com/ss-harmony-termos-green-...,"Senses Story, HARMONY, termos, zieleń",7.92,"0,0",(0),Empik,do 14 dni,Przewidywana wysyłka w 1 dzień rob.,"15,99 zł","['2025-04-16', '11:49:30']"
2,"https://www.empik.com/ss-harmony-termos-empik,...","Senses Story, ENERGY, termos, żółty",7.92,"0,0",(0),Empik,do 14 dni,Przewidywana wysyłka w 1 dzień rob.,"15,99 zł","['2025-04-16', '11:49:30']"
3,https://www.empik.com/gerlach-zestaw-nozy-w-bl...,Zestaw noży w bloku Gerlach Smart Grey 5 eleme...,24.48,"4,9",(119),NaN,do 14 dni,Przewidywana wysyłka w 1 dzień rob.,-,"['2025-04-16', '11:49:30']"
4,https://www.empik.com/filizanka-ze-spodkiem-i-...,Filiżanka Ze Spodkiem I Łyżeczką,19.11,"5,0",(3),Empik,do 14 dni,Przewidywana wysyłka w 2-3 dni rob.,"15,99 zł","['2025-04-16', '11:49:30']"
...,...,...,...,...,...,...,...,...,...,...
498,https://www.empik.com/nike-park-vii-t-shirt-63...,Nike Park VII t-shirt 635 : Rozmiar - XXL,16.02,"0,0",(0),NaN,do 14 dni,Przewidywana wysyłka w 2 dni rob.,"17,00 zł","['2025-04-16', '11:24:10']"
499,https://www.empik.com/t-shirt-damski-nike-dri-...,T-shirt damski nike dri-fit w nk one df ss dd0...,31.00,"0,0",(0),NaN,do 14 dni,Przewidywana wysyłka w 1 dzień rob.,-,"['2025-04-16', '11:24:10']"
500,https://www.empik.com/t-shirt-nike-m-nsw-tee-o...,T-shirt nike m nsw tee oc pk1 hf0469 063 r-xxl,29.84,"0,0",(0),NaN,do 14 dni,Przewidywana wysyłka w 1 dzień rob.,-,"['2025-04-16', '11:24:10']"
501,https://www.empik.com/nike-meski-t-shirt-koszu...,Nike męski t-shirt koszulka żółta Just Do It C...,16.08,"0,0",(0),NaN,do 14 dni,Przewidywana wysyłka w 2 dni rob.,"15,39 zł","['2025-04-16', '11:24:10']"


In [41]:
df_empik["rating"] = df_empik["rating"].str.replace(",", ".").astype(float)

In [42]:
df_empik["amount_of_reviews"] = df_empik["amount_of_reviews"].str.replace("(", "")
df_empik["amount_of_reviews"] = df_empik["amount_of_reviews"].str.replace(")", "").astype(int)

In [43]:
df_empik["return_policy"] = df_empik["return_policy"].str.replace("do", "")
df_empik["return_policy"] = df_empik["return_policy"].str.replace("dni", "days")

In [44]:
df_empik

,link,name,price_eur,rating,amount_of_reviews,seller,return_policy,delivery_days,shipping_eur,date_of_scrape
0,https://www.empik.com/rockle-butelka-tritanowa...,ROCKLE BUTELKA TRITANOWA KOSMOS,9.32,0.0,0,Empik,14 days,Przewidywana wysyłka w 1 dzień rob.,"15,99 zł","['2025-04-16', '11:49:30']"
1,https://www.empik.com/ss-harmony-termos-green-...,"Senses Story, HARMONY, termos, zieleń",7.92,0.0,0,Empik,14 days,Przewidywana wysyłka w 1 dzień rob.,"15,99 zł","['2025-04-16', '11:49:30']"
2,"https://www.empik.com/ss-harmony-termos-empik,...","Senses Story, ENERGY, termos, żółty",7.92,0.0,0,Empik,14 days,Przewidywana wysyłka w 1 dzień rob.,"15,99 zł","['2025-04-16', '11:49:30']"
3,https://www.empik.com/gerlach-zestaw-nozy-w-bl...,Zestaw noży w bloku Gerlach Smart Grey 5 eleme...,24.48,4.9,119,NaN,14 days,Przewidywana wysyłka w 1 dzień rob.,-,"['2025-04-16', '11:49:30']"
4,https://www.empik.com/filizanka-ze-spodkiem-i-...,Filiżanka Ze Spodkiem I Łyżeczką,19.11,5.0,3,Empik,14 days,Przewidywana wysyłka w 2-3 dni rob.,"15,99 zł","['2025-04-16', '11:49:30']"
...,...,...,...,...,...,...,...,...,...,...
498,https://www.empik.com/nike-park-vii-t-shirt-63...,Nike Park VII t-shirt 635 : Rozmiar - XXL,16.02,0.0,0,NaN,14 days,Przewidywana wysyłka w 2 dni rob.,"17,00 zł","['2025-04-16', '11:24:10']"
499,https://www.empik.com/t-shirt-damski-nike-dri-...,T-shirt damski nike dri-fit w nk one df ss dd0...,31.00,0.0,0,NaN,14 days,Przewidywana wysyłka w 1 dzień rob.,-,"['2025-04-16', '11:24:10']"
500,https://www.empik.com/t-shirt-nike-m-nsw-tee-o...,T-shirt nike m nsw tee oc pk1 hf0469 063 r-xxl,29.84,0.0,0,NaN,14 days,Przewidywana wysyłka w 1 dzień rob.,-,"['2025-04-16', '11:24:10']"
501,https://www.empik.com/nike-meski-t-shirt-koszu...,Nike męski t-shirt koszulka żółta Just Do It C...,16.08,0.0,0,NaN,14 days,Przewidywana wysyłka w 2 dni rob.,"15,39 zł","['2025-04-16', '11:24:10']"


In [45]:
df_empik["shipping_eur"] = df_empik["shipping_eur"].str.replace(",", ".")
df_empik["shipping_eur"] = df_empik["shipping_eur"].str.replace("zł", "")
df_empik["shipping_eur"] = df_empik["shipping_eur"].apply(pln_eur_converter)

In [46]:
df_empik

,link,name,price_eur,rating,amount_of_reviews,seller,return_policy,delivery_days,shipping_eur,date_of_scrape
0,https://www.empik.com/rockle-butelka-tritanowa...,ROCKLE BUTELKA TRITANOWA KOSMOS,9.32,0.0,0,Empik,14 days,Przewidywana wysyłka w 1 dzień rob.,3.73,"['2025-04-16', '11:49:30']"
1,https://www.empik.com/ss-harmony-termos-green-...,"Senses Story, HARMONY, termos, zieleń",7.92,0.0,0,Empik,14 days,Przewidywana wysyłka w 1 dzień rob.,3.73,"['2025-04-16', '11:49:30']"
2,"https://www.empik.com/ss-harmony-termos-empik,...","Senses Story, ENERGY, termos, żółty",7.92,0.0,0,Empik,14 days,Przewidywana wysyłka w 1 dzień rob.,3.73,"['2025-04-16', '11:49:30']"
3,https://www.empik.com/gerlach-zestaw-nozy-w-bl...,Zestaw noży w bloku Gerlach Smart Grey 5 eleme...,24.48,4.9,119,NaN,14 days,Przewidywana wysyłka w 1 dzień rob.,-,"['2025-04-16', '11:49:30']"
4,https://www.empik.com/filizanka-ze-spodkiem-i-...,Filiżanka Ze Spodkiem I Łyżeczką,19.11,5.0,3,Empik,14 days,Przewidywana wysyłka w 2-3 dni rob.,3.73,"['2025-04-16', '11:49:30']"
...,...,...,...,...,...,...,...,...,...,...
498,https://www.empik.com/nike-park-vii-t-shirt-63...,Nike Park VII t-shirt 635 : Rozmiar - XXL,16.02,0.0,0,NaN,14 days,Przewidywana wysyłka w 2 dni rob.,3.96,"['2025-04-16', '11:24:10']"
499,https://www.empik.com/t-shirt-damski-nike-dri-...,T-shirt damski nike dri-fit w nk one df ss dd0...,31.00,0.0,0,NaN,14 days,Przewidywana wysyłka w 1 dzień rob.,-,"['2025-04-16', '11:24:10']"
500,https://www.empik.com/t-shirt-nike-m-nsw-tee-o...,T-shirt nike m nsw tee oc pk1 hf0469 063 r-xxl,29.84,0.0,0,NaN,14 days,Przewidywana wysyłka w 1 dzień rob.,-,"['2025-04-16', '11:24:10']"
501,https://www.empik.com/nike-meski-t-shirt-koszu...,Nike męski t-shirt koszulka żółta Just Do It C...,16.08,0.0,0,NaN,14 days,Przewidywana wysyłka w 2 dni rob.,3.59,"['2025-04-16', '11:24:10']"


In [47]:
df_empik["delivery_days"] = df_empik["delivery_days"].str.replace("Przewidywana wysyłka w", "")
df_empik["delivery_days"] = df_empik["delivery_days"].str.replace("dzień rob.", "")
df_empik["delivery_days"] = df_empik["delivery_days"].str.replace("dni rob.", "")
df_empik["delivery_days"] = df_empik["delivery_days"].str.split("-").str[0]
df_empik["delivery_days"] = pd.to_numeric(df_empik["delivery_days"], errors = "coerce")

In [48]:
df_empik["date_of_scrape"] = df_empik["date_of_scrape"].str.extract(r"'(\d{4}-\d{2}-\d{2})'")
df_empik["scraping_date"] = pd.to_datetime(df_empik["date_of_scrape"], errors="coerce")
df_empik.drop(columns="date_of_scrape", inplace=True)

In [49]:
df_empik

,link,name,price_eur,rating,amount_of_reviews,seller,return_policy,delivery_days,shipping_eur,scraping_date
0,https://www.empik.com/rockle-butelka-tritanowa...,ROCKLE BUTELKA TRITANOWA KOSMOS,9.32,0.0,0,Empik,14 days,1.0,3.73,2025-04-16
1,https://www.empik.com/ss-harmony-termos-green-...,"Senses Story, HARMONY, termos, zieleń",7.92,0.0,0,Empik,14 days,1.0,3.73,2025-04-16
2,"https://www.empik.com/ss-harmony-termos-empik,...","Senses Story, ENERGY, termos, żółty",7.92,0.0,0,Empik,14 days,1.0,3.73,2025-04-16
3,https://www.empik.com/gerlach-zestaw-nozy-w-bl...,Zestaw noży w bloku Gerlach Smart Grey 5 eleme...,24.48,4.9,119,NaN,14 days,1.0,-,2025-04-16
4,https://www.empik.com/filizanka-ze-spodkiem-i-...,Filiżanka Ze Spodkiem I Łyżeczką,19.11,5.0,3,Empik,14 days,2.0,3.73,2025-04-16
...,...,...,...,...,...,...,...,...,...,...
498,https://www.empik.com/nike-park-vii-t-shirt-63...,Nike Park VII t-shirt 635 : Rozmiar - XXL,16.02,0.0,0,NaN,14 days,2.0,3.96,2025-04-16
499,https://www.empik.com/t-shirt-damski-nike-dri-...,T-shirt damski nike dri-fit w nk one df ss dd0...,31.00,0.0,0,NaN,14 days,1.0,-,2025-04-16
500,https://www.empik.com/t-shirt-nike-m-nsw-tee-o...,T-shirt nike m nsw tee oc pk1 hf0469 063 r-xxl,29.84,0.0,0,NaN,14 days,1.0,-,2025-04-16
501,https://www.empik.com/nike-meski-t-shirt-koszu...,Nike męski t-shirt koszulka żółta Just Do It C...,16.08,0.0,0,NaN,14 days,2.0,3.59,2025-04-16


In [50]:
scraped_df.drop(columns="delivery_clean", inplace=True)

In [51]:
df_empik["platform"] = "empik"
df_empik['bought_so_far'] = None

In [52]:
print(scraped_df.columns)
print(df_empik.columns)

Index(['name', 'price_eur', 'rating', 'amount_of_reviews', 'link',
       'bought_so_far', 'seller', 'return_policy', 'platform', 'scraping_date',
       'delivery_days', 'shipping_eur'],
      dtype='object')
Index(['link', 'name', 'price_eur', 'rating', 'amount_of_reviews', 'seller',
       'return_policy', 'delivery_days', 'shipping_eur', 'scraping_date',
       'platform', 'bought_so_far'],
      dtype='object')


In [53]:
df = pd.concat([scraped_df, df_empik], ignore_index=True)

In [54]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [55]:
df

,name,price_eur,rating,amount_of_reviews,link,bought_so_far,seller,return_policy,platform,scraping_date,delivery_days,shipping_eur
0,Szkło ochronne do Nikon D7500 - 3mk Cam Protec...,6.52,0.0,0,https://www.empik.com/3mk-nikon-d7500-cam-prot...,None,Empik,14 days,empik,2025-04-16,3.0,3.73
1,D7100 24.1 MP DX-Format CMOS Digital SLR with ...,849.00,4.4,335,https://www.amazon.com/Nikon-DX-Format-Digital...,No buyers,Willoughby's Established 1898,30 days,amazon,2025-04-14,18.0,117.29
2,Soft Extended Eyecup Eyepiece Eyeshade for Nik...,13.99,4.4,120,https://www.amazon.com/Extended-Eyepiece-Eyesh...,No buyers,MagiCube Photography,30 days,amazon,2025-04-14,16.0,8.43
3,Nikon D7500 Body 77422,712.23,0.0,0,https://www.ebay.com/itm/306227359747?_skw=Nik...,No buyers,digree_japan,30 days,ebay,2025-04-14,10.0,30.0
4,Mens Challenge Iv Soccer Jersey,23.99,0.0,0,https://www.amazon.com/Nike-Dri-Fit-Challenge-...,No buyers,NaN,None,amazon,2025-04-14,16.0,No information provided
...,...,...,...,...,...,...,...,...,...,...,...,...
1814,Courage The Cowardly Dog Creepy Stuff Happens ...,11.99,0.0,0,https://www.ebay.com/itm/276816672682?_skw=kit...,No buyers,pokeforcegold,30 days,ebay,2025-04-14,28.0,12.52
1815,Nikon D7500 DX-Format Digital SLR Body - Black...,750.00,5.0,52,https://www.ebay.com/itm/115987401833?_skw=Nik...,No buyers,dbrowers,no returns,ebay,2025-04-14,NaN,No information provided
1816,"Camera Backpack, DSLR/SLR/Mirrorless Tactical ...",64.59,4.5,700,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,300,Mosiso,30 days,amazon,2025-04-14,18.0,24.08
1817,"Dzbanek na podgrzewaczu TADAR, biały, 1 l",19.58,5.0,1,https://www.empik.com/dzbanek-na-podgrzewaczu-...,None,NaN,14 days,empik,2025-04-16,1.0,NaN


In [57]:
def assign_category(name):
    name = str(name).lower()

    if any(word in name for word in ["nikon", "lens", "tripod", "camera", "dslr", "bag", "strap", "obiektyw", "D7500", "digital"]):
        return "camera"
    elif any(word in name for word in [
        "mug", "cup", "bottle", "pan", "pot", "knife", "fork", "spoon", "pieprz", "młynek", "stojak", "karafka", "meat", "maselniczka", "butelka"
        "plate", "kitchen", "garnek", "patelnia", "termos", "czajnik", "kubek", "dzbanek", "cukiernica", "lunchbox", "BBQ", "kamienie", "salt"]):
        return "kitchen_accessories"
    elif any(word in name for word in ["shirt", "t-shirt", "koszulka", "jersey", "sleeve", "nike", "Sportswear"]):
        return "tshirts"
    else:
        return "other"

df["Category"] = df["name"].apply(assign_category)

In [58]:
df['Category'].value_counts()

Category
tshirts                634
kitchen_accessories    542
camera                 468
other                  175
Name: count, dtype: int64

In [ ]:
df = df[df['Category'] != 'other']

drop_kitchen = df[(df['Category'] == 'kitchen_accessories') & (df['platform'] == 'amazon')].sample(n=125, random_state=42).index
drop_tshirts = df[(df['Category'] == 'tshirts') & (df['platform'] == 'amazon')].sample(n=125, random_state=42).index

df = df.drop(drop_kitchen.union(drop_tshirts))

df = df.reset_index(drop=True)

In [60]:
df

,name,price_eur,rating,amount_of_reviews,link,bought_so_far,seller,return_policy,platform,scraping_date,delivery_days,shipping_eur,Category
0,Szkło ochronne do Nikon D7500 - 3mk Cam Protec...,6.52,0.0,0,https://www.empik.com/3mk-nikon-d7500-cam-prot...,None,Empik,14 days,empik,2025-04-16,3.0,3.73,camera
1,D7100 24.1 MP DX-Format CMOS Digital SLR with ...,849.00,4.4,335,https://www.amazon.com/Nikon-DX-Format-Digital...,No buyers,Willoughby's Established 1898,30 days,amazon,2025-04-14,18.0,117.29,camera
2,Soft Extended Eyecup Eyepiece Eyeshade for Nik...,13.99,4.4,120,https://www.amazon.com/Extended-Eyepiece-Eyesh...,No buyers,MagiCube Photography,30 days,amazon,2025-04-14,16.0,8.43,camera
3,Nikon D7500 Body 77422,712.23,0.0,0,https://www.ebay.com/itm/306227359747?_skw=Nik...,No buyers,digree_japan,30 days,ebay,2025-04-14,10.0,30.0,camera
4,Mens Challenge Iv Soccer Jersey,23.99,0.0,0,https://www.amazon.com/Nike-Dri-Fit-Challenge-...,No buyers,NaN,None,amazon,2025-04-14,16.0,No information provided,tshirts
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,Courage The Cowardly Dog Creepy Stuff Happens ...,11.99,0.0,0,https://www.ebay.com/itm/276816672682?_skw=kit...,No buyers,pokeforcegold,30 days,ebay,2025-04-14,28.0,12.52,kitchen_accessories
1390,Nikon D7500 DX-Format Digital SLR Body - Black...,750.00,5.0,52,https://www.ebay.com/itm/115987401833?_skw=Nik...,No buyers,dbrowers,no returns,ebay,2025-04-14,NaN,No information provided,camera
1391,"Camera Backpack, DSLR/SLR/Mirrorless Tactical ...",64.59,4.5,700,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,300,Mosiso,30 days,amazon,2025-04-14,18.0,24.08,camera
1392,"Dzbanek na podgrzewaczu TADAR, biały, 1 l",19.58,5.0,1,https://www.empik.com/dzbanek-na-podgrzewaczu-...,None,NaN,14 days,empik,2025-04-16,1.0,NaN,kitchen_accessories


In [61]:
df.dtypes

name                         object
price_eur                   float64
rating                      float64
amount_of_reviews             int64
link                         object
bought_so_far                object
seller                       object
return_policy                object
platform                     object
scraping_date        datetime64[ns]
delivery_days               float64
shipping_eur                 object
Category                     object
dtype: object

In [62]:
df["shipping_eur"] = pd.to_numeric(df["shipping_eur"], errors="coerce")

In [63]:
df.dtypes

name                         object
price_eur                   float64
rating                      float64
amount_of_reviews             int64
link                         object
bought_so_far                object
seller                       object
return_policy                object
platform                     object
scraping_date        datetime64[ns]
delivery_days               float64
shipping_eur                float64
Category                     object
dtype: object

In [64]:
df.to_csv("df.csv", index=False)